# Micrograd: Tutorial on automatic differentiation using backpropagation
## by Andrej Karpathy

This notebook just contains the code that he copy-pastes and that is too long to type along, along with a time stamp referring to [the video](https://www.youtube.com/watch?v=VMj-3S1tku0).
You will need to change this code later, so it is easiest to follow along by just adding to this notebook.

# Preliminary exploration

## Required modules

Dependencies for math operations and plotting

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Functions of a single variable

In [10]:
def f(x):
    return 3.0*x**2 - 4.0*x + 5.0

In [11]:
f(3.0)

20.0

In [ ]:
xs = np.arange(-5, 5, 0.25)
ys = f(xs)
plt.plot(xs, ys)

### Numerical derivative

In [ ]:
h = 0.001
x = 3.0
f(x + h)  # fill in

## Functions of multiple variables

In [14]:
# Let's get more complex

# Inputs
a = 2.0
b = -3.0
c = 10.0

# Function
d = a*b + c
print(d)

4.0


### Numerical derivative

In [ ]:
h = 0.0001

# Inputs
a = 2.0
b = -3.0
c = 10.0


# Function with variations
d1 = a*b + c
a += h
d2 = a*b + c

print('d1', d1)
print('d2', d2)
# print('slope', ...)  # fill in

# Micrograd

## Value class

In [ ]:
class Value:
    
    def __init__(self, data):
        self.data = data
        
    def __repr__(self):
        # Print a nice output
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        # Addition operator
        out = Value(self.data + other.data)
        return out
        
    def __mul__(self, other):
        # Multiplication operator
        out = Value(self.data * other.data)
        
#     def tanh(self):
#         # Hyperbolic tangent function
#         x = self.data
#         t = (math.exp(2.0*x) - 1.0)/(math.exp(2.0*x) + 1)
#         out = Value(t, (self, ), 'tanh')
#         return out

In [24]:
a = Value(2.0)
b = Value(-3.0)
print(a)
print(b)

Value(data=2.0)
Value(data=-3.0)


## Visualization code (at time 25:07)

Copy pasted code with visualization functions

In [2]:
from graphviz import Digraph

def trace(root):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})  # LR = left to right
    
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name = uid, label = "{ data %.4f }" % (n.data, ), shape='record')
        if n._op:
            # if this value is a result of some operation, create an op node for it
            dot.node(name = uid + n._op, label = n._op)
            # and connect this node to it
            dot.edge(uid + n._op, uid)
            
    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
            
    return dot

You can now visualize the ``Value`` objects

In [ ]:
draw_dot(d)

## Basic Neuron

In [ ]:
# Input x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# Weights w1, w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# Bias of the neuron
b = Value(6.7, label='b')

# Computation of x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label='x1*w1' 
x2w2 = x2*w2; x2w2.label='x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x2*w1 + x2*w2'
n = x1w1x2w2 + b; n.label='n'
draw_dot(n)

# o = n.tanh(); o.label='o'  # you first need to uncomment tanh in the definition of Value
# draw_dot(o)

## Topological sort code (at 1:18:45 )

Function that constructs the acyclical graph of the chain of operations such that we can start at the top and traverse each edge of the graph once.

In [ ]:
# Define variable o first before running what follows

In [ ]:
topo = []
visited = set()
def build_topo(v):
    if v not in visited:
        visited.add(v)
        for child in v._prev:
            build_topo(child)
        topo.append(v)
build_topo(o)
topo

In [ ]:
o.grad = 1.0 # initialize the gradient of the last node

# Initialize the gradients in the order given by the topological ordering
for node in reversed(topo):
    node._backward()
    
draw_dot(o)

## 1:39:54 Pytorch comparison

In [ ]:
import torch

x1 = torch.Tensor([2.0]).double()                ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()                ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double()               ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()                ; w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double()  ; b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

## 1:44:37 Neuron

In [ ]:
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))

## 1:48:22 Layer

In [ ]:
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs

## 1:49:19 MLP

In [ ]:
class MLP:
    
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

## 1:51:07 Example dataset

In [ ]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0]  # desired targets